In [1]:
from pyemma.msm import MaximumLikelihoodMSM, BayesianMSM, MaximumLikelihoodHMSM, its
from bhmm import lag_observations
import pyemma.plots as mplt
from msmbuilder.cluster import NDGrid
from sklearn.pipeline import Pipeline
import pickle
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.stats import entropy
import pandas as pd

/Users/robert_arbon/anaconda/envs/sonification/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/robert_arbon/anaconda/envs/sonification/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/robert_arbon/anaconda/envs/sonification/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version.

In [2]:
xmin, xmax = -1.2, 1.2
tau = 8
data_name = 'four-well'
n = 4

## Load data and cluster

In [3]:
X = [np.load(x) for x in glob('data/'+data_name+'/*npy')]
# X = [y for x in X for y in np.array_split(x,8)]
print(X[0].shape)

(845, 1)


In [56]:
m_opt = min(200,int(np.sqrt(len(X)*X[0].shape[0])))
print(m_opt)
cluster = NDGrid(min=xmin, max=xmax, n_bins_per_feature=m_opt)
dtrajs_all = cluster.fit_transform(X)

200


## MSM

In [207]:
m = MaximumLikelihoodMSM(lag=tau, connectivity='largest', reversible=True )
dtrajs = [dtrajs_all[i][:80] for i in range(int(len(dtrajs_all)/2))]
m.fit(dtrajs)


MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=8, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=10,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

In [208]:
print(len(m.active_set))
print(m.active_count_fraction)
print(m.active_state_fraction)
m.active_set

169
0.99975
0.8894736842105263


array([ 15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 187, 189])

In [209]:
m.discrete_trajectories_active[:2]

[array([100, 108, 109, 103,  99, 100, 109, 111, 107, 102, 102,  98, 102,
        104, 111, 109, 103, 101, 100,  96,  90,  90,  97,  94,  94,  93,
         95,  96,  94,  97, 102, 103,  95, 101, 107, 104,  97,  89,  99,
         99, 103, 110, 105, 112, 117, 111, 110, 106, 107, 105, 106, 113,
        108, 106, 109, 110, 115, 117, 115, 109, 106, 111, 106, 110, 106,
        102, 109, 111, 109, 115, 117, 124, 121, 121, 123, 126, 128, 131,
        131, 133]),
 array([ 90,  94,  97,  98,  96,  96,  93,  91,  89,  89,  88,  91,  89,
         94,  97,  99,  96,  88,  89,  89,  89,  87,  86,  90,  92,  94,
         95,  97,  97, 101, 101, 108, 104, 102, 102, 105, 106, 105, 105,
        104, 106, 103,  97,  95, 101,  96, 101, 107, 107, 109, 110, 108,
        107, 112, 112, 111, 107, 108, 105, 106, 104, 107, 111, 106, 104,
        115, 110, 108, 108, 111, 115, 117, 119, 118, 118, 112, 110, 105,
        105, 106])]

In [210]:
np.sum(m.count_matrix_active)

3599.0

In [217]:
n_obs = np.sum([len(np.where(x>-1)[0]) for x in m.discrete_trajectories_active])
n_obs_full = np.sum([len(x) for x in m.discrete_trajectories_full])
print(n_obs, n_obs_full)


3999 4000


## HMM

In [267]:
hmm = MaximumLikelihoodHMSM(nstates=4, lag=tau, connectivity='largest', reversible=True, msm_init=m)
hmm.fit(dtrajs)

MaximumLikelihoodHMSM(accuracy=0.001, connectivity='largest',
           dt_traj='1 step', lag=8, maxit=1000, mincount_connectivity=0.25,
           msm_init=MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=8, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=10,
           score_method='VAMP2', sparse=False, statdist_constraint=None),
           nstates=2, observe_nonempty=True, reversible=True,
           separate=None, stationary=False, stride=1)

In [268]:
hmm.P

array([[ 0.92637221,  0.07362779],
       [ 0.02604168,  0.97395832]])

In [269]:
hmm.dtrajs_obs[:2]

[array([100, 108, 109, 103,  99, 100, 109, 111, 107, 102, 102,  98, 102,
        104, 111, 109, 103, 101, 100,  96,  90,  90,  97,  94,  94,  93,
         95,  96,  94,  97, 102, 103,  95, 101, 107, 104,  97,  89,  99,
         99, 103, 110, 105, 112, 117, 111, 110, 106, 107, 105, 106, 113,
        108, 106, 109, 110, 115, 117, 115, 109, 106, 111, 106, 110, 106,
        102, 109, 111, 109, 115, 117, 124, 121, 121, 123, 126, 128, 131,
        131, 133]),
 array([ 90,  94,  97,  98,  96,  96,  93,  91,  89,  89,  88,  91,  89,
         94,  97,  99,  96,  88,  89,  89,  89,  87,  86,  90,  92,  94,
         95,  97,  97, 101, 101, 108, 104, 102, 102, 105, 106, 105, 105,
        104, 106, 103,  97,  95, 101,  96, 101, 107, 107, 109, 110, 108,
        107, 112, 112, 111, 107, 108, 105, 106, 104, 107, 111, 106, 104,
        115, 110, 108, 108, 111, 115, 117, 119, 118, 118, 112, 110, 105,
        105, 106])]

In [270]:
n_obs = np.sum([len(np.where(x>-1)[0]) for x in hmm.dtrajs_obs])
n_obs

3999

In [271]:
n_h_obs = np.sum([len(np.where(x>-1)[0]) for x in hmm.hidden_state_trajectories])
n_h_obs


4000

In [274]:
int(np.sum(hmm.count_matrix))

1728

In [276]:
hmm.msm_init.active_count_fraction

0.99975